GoogleDriveに接続

In [0]:
from google.colab import drive
drive.mount('/content/drive')

ライブラリの読み込み

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Input, Flatten, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.utils.np_utils import to_categorical
import numpy as np
import os

画像の読み込み

In [0]:
image_list = []
label_list = []
input_size = 64

for dir in os.listdir('drive/My Drive/Colab Notebooks/image/train'):
    dir1 = "drive/My Drive/Colab Notebooks/image/train/" + dir 
    label = 0

    if dir == "Gotsu_shio" :        #マルちゃん ごつ盛り 塩焼そば
        label = 0
    elif dir == "Gotsu_source" :    #マルちゃん ごつ盛り ソース焼そば
        label = 1
    elif dir == "Bubuka" :          #明星 ぶぶか 油そば
        label = 2
    elif dir == "Ippei" :           #明星 一平ちゃん夜店の焼そば
        label = 3
    elif dir == "UFO" :             #日清 焼そばU.F.O.
        label = 4

    for file in os.listdir(dir1):
        label_list.append(label)
        filepath = dir1 + "/" + file
        img = img_to_array(load_img(filepath, target_size=(input_size,input_size)))
        image_list.append(img)

#numpy配列に変換
image_list = np.array(image_list)
print(image_list.shape)

#ラベルをOne-hot表現に変換
Y = to_categorical(label_list)

ネットワークの構築，学習

In [0]:
# モデルを生成してニューラルネットを構築
model = Sequential()

model.add(Conv2D(32,(3,3), padding = "same", input_shape =(input_size, input_size, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32,(3,3), padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3), padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64,(3,3), padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))


model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(128,(3,3), padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(128,(3,3), padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))


model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.4))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(.50))

model.add(Dense(5))
model.add(Activation("softmax"))

opt = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

#モデルのコンパイル
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()
#学習を実行，10%はテストに使用
model.fit(image_list, Y, epochs=20, batch_size=100, validation_split=0.1, shuffle = True)

推論

In [0]:
#推論
total = 0.
correct = 0.

for dir in os.listdir("drive/My Drive/Colab Notebooks/image/test"):

    dir1 = "drive/My Drive/Colab Notebooks/image/test/" + dir 
    label = 0

    if dir == "Gotsu_shio":
        label = 0
    elif dir == "Gotsu_source":
        label = 1
    elif dir == "Bubuka":
        label = 2
    elif dir == "Ippei":
        label = 3
    elif dir == "UFO":
        label = 4
        
    for file in os.listdir(dir1):
        label_list.append(label)
        filepath = dir1 + "/" + file
        print(filepath)
        img = []
        img.append(img_to_array(load_img(filepath, target_size=(input_size,input_size))))
        result = model.predict_classes(np.array(img))
        print("label:", label, "result:", result[0])

        total += 1.

        if label == result[0]:
            correct += 1.

print("正解率: ", correct / total * 100, "%")

model.save("drive/My Drive/Colab Notebooks/model.h5f", overwrite=True)